## Reducing Astronomical CCD Data

The calibration of astronomical data from CCDs requires that you account for any readout bias from your detector, the background dark current (the signal measured with the aperture closed), as well as adjust for the detector sensitivity (typically called a flat-field). This notebook is designed to illustrate that process for an example dataset, in a folder you will make called ''ccd-intro-data''. This notebook has been adapted from Example 1 in the [CCD Data Reduction Guide] (https://www.astropy.org/ccd-reduction-and-photometry-guide/v/dev/notebooks/06-03-science-images-calibration-examples.html) 

First, let's load our Python packages. 

In [80]:
from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np

from astropy import units as u
from astropy.nddata import CCDData
from astropy.stats import mad_std
import ccdproc as ccdp

Next, let's define a function that will check for the best exposure to use given a FITS image and a list of dark exposure times.

In [81]:
def find_nearest_dark_exposure(image, dark_exposure_times, tolerance=0.5):
    """
    Find the nearest exposure time of a dark frame to the exposure time of the image,
    raising an error if the difference in exposure time is more than tolerance.
    
    Parameters
    ----------
    
    image : astropy.nddata.CCDData
        Image for which a matching dark is needed.
    
    dark_exposure_times : list
        Exposure times for which there are darks.
    
    tolerance : float or ``None``, optional
        Maximum difference, in seconds, between the image and the closest dark. Set
        to ``None`` to skip the tolerance test.
    
    Returns
    -------
    
    float
        Closest dark exposure time to the image.
    """

    dark_exposures = np.array(list(dark_exposure_times))
    idx = np.argmin(np.abs(dark_exposures - image.header['exptime']))
    closest_dark_exposure = dark_exposures[idx]

    if (tolerance is not None and 
        np.abs(image.header['exptime'] - closest_dark_exposure) > tolerance):
        
        raise RuntimeError('Closest dark exposure time is {} for flat of exposure '
                           'time {}.'.format(closest_dark_exposure, a_flat.header['exptime']))
        
    
    return closest_dark_exposure

Next, let's define our data folder create ImageFileCollections for the raw data and the calibrated data, and define variables for the image type of the flat field and the science images (there are, unfortunately, no standard names for these, so double check your FITS header first).



In [82]:
reduced_path = Path('../../ccd-intro-data/SOAR_3I_2025_708_reduced')

science_imagetyp = '3IATLAS'
#Update these with the exact name that the TO gives to the flat field
flat_imagetyp = 'flatfield'
bias_imagetyp = 'BIAS'
dark_imagetyp = 'DARK'
exposure = 'exptime'

#ifc_reduced = ccdp.ImageFileCollection(reduced_path)
ifc_raw = ccdp.ImageFileCollection('../../ccd-intro-data/SOAR_3I_20250708_raw',keywords=['*'])

Next, a quick look at the science images in this data set.

In [83]:
lights = ifc_raw.summary[ifc_raw.summary['object'] == science_imagetyp.upper()]
lights['date-obs', 'file', 'object', 'filter', 'exptime']

date-obs,file,object,filter,exptime
str23,str26,str7,str6,float64
2025-07-09T04:43:18.384,0040_3I_atlas.fits,3IATLAS,i-SDSS,60.0
2025-07-09T04:46:28.807,0041_3I_atlas.fits,3IATLAS,i-SDSS,60.0
2025-07-09T04:59:32.738,0042_3I_atlas.fits,3IATLAS,i-SDSS,60.0
2025-07-09T05:02:01.990,0043_3I_atlas.fits,3IATLAS,i-SDSS,60.0
2025-07-09T05:05:33.023,0044_3I_atlas.fits,3IATLAS,i-SDSS,60.0
2025-07-09T05:12:46.700,0045_3I_atlas.fits,3IATLAS,i-SDSS,30.0
...,...,...,...,...
2025-07-09T05:58:26.424,0099_3I_atlas_r.fits,3IATLAS,r-SDSS,30.0
2025-07-09T05:59:07.542,0100_3I_atlas_r.fits,3IATLAS,r-SDSS,30.0


# Creating your combined bias


In [116]:
bias_raw_path = Path('../../ccd-intro-data/SOAR_3I_20250709_raw/bias')
bias_reduced_path = Path('../../ccd-intro-data/SOAR_3I_2025_0708_reduced/bias/')

ifc_bias_raw = ccdp.ImageFileCollection(bias_raw_path,keywords=['*'])

#Note that this loop is different than some of the others below. With SOAR data I was having trouble getting the 
#code to actually read the CCD files using the IFC.ccds function, so I had to do it more explicitly like this. 
#This method also has the added benefit of adding a new unit to the header. 
for file_name in ifc_bias_raw.files_filtered(obstype='BIAS'):           ## Just get the bias frames Provide the file name too.
    ccd = CCDData.read(str(ifc_bias_raw.location)+'/' + file_name, unit = u.adu) 
    # Trim the overscan, if needed. As is does not trim anything. 
    
    #ccd = ccdp.trim_image(ccd[:, :])
    
    # Save the result
    ccd.write(bias_reduced_path / file_name, overwrite=True)


In [114]:
ifc_bias_reduced = ccdp.ImageFileCollection(bias_reduced_path,keywords=['*'])
calibrated_biases = ifc_bias_reduced.files_filtered(obstype='bias', include_path=True)

combined_bias = ccdp.combine(calibrated_biases,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std,
                             mem_limit=350e6,
                            unit='adu')

combined_bias.meta['combined'] = True

combined_bias.write(bias_reduced_path / 'combined_bias.fit')

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:splitting each image into 6 chunks to limit memory usage to 350000000.0 bytes.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: splitting each image into 6 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


# Reducing Your Dark and Flat Fields


First we need to reduce our dark files by removing the bias and saving them into our reduced directory

In [46]:
#ifc_darks_raw = ccdp.ImageFileCollection('../../ccd-intro-data/SOAR_3I_20250708_raw/darks')
#ifc_darks_raw.summary['file', 'imagetyp', 'exptime', 'filter']

ifc_bias_reduced = ccdp.ImageFileCollection(bias_reduced_path)
combined_bias = CCDData.read(ifc_bias_raw.files_filtered(obstype='BIAS', 
                                                               combined=True, 
                                                               include_path=True)[0],unit='adu')

for ccd, file_name in ifc_darks_raw.ccds(imagetyp='DARK',            # Just get the bias frames
                                          return_fname=True           # Provide the file name too.
                                         ):
        
    # Trim the overscan
    ccd = ccdp.trim_image(ccd[:, :4096])
    
    # Subtract bias
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    # Save the result
    ccd.write(ex2_path_reduced / file_name)
    ifc_reduced = ccdp.ImageFileCollection(reduced_path)
    ifc_reduced.refresh()
    ifc_reduced.summary['file', 'imagetyp', 'exptime', 'filter', 'combined']

IndexError: list index out of range

Then we can do the same for our flat field images, which need dark and bias correction.

In [ ]:
combined_dark_files = ifc_reduced.files_filtered(imagetyp='dark', combined=True)

#Update this with the keyword that the SOAR TO uses
flat_image_type = 'FLATFIELD'

#Check to make sure there are enough dark files to accurately run

n_combined_dark = len(combined_dark_files)
expected_exposure_times = set([7, 70, 300])

if n_combined_dark < 3:
    raise RuntimeError('One or more combined dark is missing. Please re-run the dark notebook.')
elif n_combined_dark > 3:
    raise RuntimeError('There are more combined dark frames than expected.')
    
actual_exposure_times = set(h['exptime'] for h in ifc_reduced.headers(imagetyp='dark', combined=True))

if (expected_exposure_times - actual_exposure_times):
    raise RuntimeError('Encountered unexpected exposure time in combined darks. '
                       'The unexpected times are {}'.format(actual_exposure_times - expected_exposure_times))
    

Set up the loop to reduce all of the available flats.

In [ ]:
for ccd, file_name in ifc_raw.ccds(imagetyp='FLAT',            # Just get the bias frames
                                   return_fname=True           # Provide the file name too.
                                  ):
        
    # Trim the overscan
    ccd = ccdp.trim_image(ccd[:, :4096])
    
    # Find the correct dark exposure
    closest_dark = find_nearest_dark_exposure(ccd, actual_exposure_times, tolerance=100)
    
    # Subtract the dark current 
    ccd = ccdp.subtract_dark(ccd, combined_darks[closest_dark],
                             exposure_time='exptime', exposure_unit=u.second, scale=True)

    # Save the result
    ccd.write(reduced_path / file_name)

# Setting up for Bias, Dark, and Flat Reduction

The next step is building your bias, dark, and flat-field files

In [ ]:
#These lines pull the calibrated bias, dark, and flat field images from the reduced folder, so make sure you have them
#reduced prior. 

combined_darks = {ccd.header[exposure]: ccd for ccd in ifc_reduced.ccds(imagetyp='dark', combined=True)}
combined_flats = {ccd.header['filter']: ccd for ccd in ifc_reduced.ccds(imagetyp=flat_imagetyp, combined=True)}

# There is only one bias, so no need to set up a dictionary.
combined_bias = [ccd for ccd in ifc_reduced.ccds(imagetyp='bias', combined=True)][0]

#Rembember to run multiple pipelines for multiple filters, each filter needs its own flatfield (one for i-SDSS and
#one for r-SDSS) 
all_images = []
light_ccds = []
for light, file_name in ifc_raw.ccds(imagetyp=science_imagetyp, return_fname=True):
    light_ccds.append(light)
    
    #Note that we may not need to trim our image, visually inspect via DS9 beforehand
    reduced = ccdp.trim_image(light[:, :4096])
    
    # Note that the first argument in the remainder of the ccdproc calls is
    # the *reduced* image, so that the calibration steps are cumulative.
    reduced = ccdp.subtract_bias(reduced, combined_bias)
    
    closest_dark = find_nearest_dark_exposure(reduced, combined_darks.keys())

    reduced = ccdp.subtract_dark(reduced, combined_darks[closest_dark], 
                                 exposure_time='exptime', exposure_unit=u.second)
    
    good_flat = combined_flats[reduced.header['filter']]
    reduced = ccdp.flat_correct(reduced, good_flat)
    all_images.append(reduced)
    reduced.write(reduced_path / file_name)